Código com algumas alterações de: https://amandeepsaluja.com/extracting-job-information-from-linkedin-jobs-using-beautifulsoup-and-selenium/

In [2]:
# importing packages
import pandas as pd
import re
import numpy as np

from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
from IPython.core.display import clear_output
from random import randint
from requests import get
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from time import time
start_time = time()

from warnings import warn

In [3]:
# replace variables here.
# Psicologia NOT ("Profissional+Liberal" OR "divulga+vagas" OR "seleção+profissional")
url = "https://www.linkedin.com/jobs/search?keywords=Psicologia%20NOT%20%28%22Profissional%2BLiberal%22%20OR%20%22divulga%2Bvagas%22%20OR%20%22sele%C3%A7%C3%A3o%2Bprofissional%22%29&location=Brasil&geoId=106057199&trk=public_jobs_jobs-search-bar_search-submit&redirect=false&position=1&pageNum=0"
no_of_jobs = 600

In [4]:
# this will open up new window with the url provided above 
driver = webdriver.Chrome(r'C:\Users\marcellohro\Downloads\chromedriver.exe')
driver.get(url)
sleep(3)
action = ActionChains(driver)

In [5]:
# load the whole page
for i in range(8):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    sleep(1)

In [6]:
# to show more jobs. Depends on number of jobs selected
i = 1
while i <= ((no_of_jobs-175)/25): 
    driver.find_element_by_xpath('/html/body/main/div/section/button').click()
    i = i + 1
    sleep(3)

In [7]:
# parsing the visible webpage
pageSource = driver.page_source
lxml_soup = BeautifulSoup(pageSource, 'lxml')

# searching for all job containers
job_container = lxml_soup.find('ul', class_ = 'jobs-search__results-list')

print('You are scraping information about {} jobs.'.format(len(job_container)))

You are scraping information about 600 jobs.


In [8]:
# setting up list for job information
job_id = []
post_title = []
company_name = []
post_date = []
job_location = []

In [9]:
# for loop for job title, company, id, location and date posted
for job in job_container:
    
    # job title
    job_titles = job.find("span", class_="screen-reader-text").text
    post_title.append(job_titles)
    
    # linkedin job id
    job_ids = job.find('a', href=True)['href']
    job_ids = re.findall(r'(?!-)([0-9]*)(?=\?)',job_ids)[0]
    job_id.append(job_ids)
    
    # company name
    company_names = job.select_one('img')['alt']
    company_name.append(company_names)
    
    # job location
    job_locations = job.find("span", class_="job-result-card__location").text
    job_location.append(job_locations)
    
    # posting date
    post_dates = job.select_one('time')['datetime']
    post_date.append(post_dates)

In [10]:
# setting up list for job information
job_desc = []
level = []
emp_type = []
functions = []
industries = []
appl_num = []

In [11]:
# for loop for job description and criterias
for x in range(1,len(job_id)+1):
    
    # clicking on different job containers to view information about the job
    job_xpath = '/html/body/main/div/section/ul/li[{}]/img'.format(x)
    driver.find_element_by_xpath(job_xpath).click()
    sleep(2)
    
    # Applicants
    try:
        applicants_xpath = '/html/body/main/section/div[2]/section[1]/div[1]/div[1]/h3[2]/span[2]'
        applicants = driver.find_element_by_xpath(applicants_xpath).text
        appl_num.append(applicants)
    except:
        appl_num.append('25')
    
    # clicking on show more
    try:
        job_xpath = '/html/body/main/section/div[2]/section[2]/div[2]/section/button[1]/icon'
        driver.find_element_by_xpath(job_xpath).click()
    except Exception:
        pass
    try:
        job_xpath = '/html/body/main/section/div[2]/section[2]/div/section/button[1]/icon'
        driver.find_element_by_xpath(job_xpath).click()
    except Exception:
        pass
    sleep(0.5)
    
            
    # job description
    try:
        jobdesc_xpath = '/html/body/main/section/div[2]/section[2]/div'
        job_descs = driver.find_element_by_xpath(jobdesc_xpath).text
        job_desc.append(job_descs)
    except:
        job_desc.append('')
        
    # job criteria container below the description
#     job_criteria_container = lxml_soup.find('ul', class_ = 'job-criteria__list')
#     all_job_criterias = job_criteria_container.find_all("span", class_='job-criteria__text job-criteria__text--criteria')
    
    # Seniority level
    try:
        seniority_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[1]'
        seniority = driver.find_element_by_xpath(seniority_xpath).text.splitlines(0)[1]
        level.append(seniority)
    except:
        level.append('')
    
    # Employment type
    try:
        type_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[2]'
        employment_type = driver.find_element_by_xpath(type_xpath).text.splitlines(0)[1]
        emp_type.append(employment_type)
    except:
        emp_type.append('')
    
    # Job function
    try:
        function_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[3]'
        job_function = driver.find_element_by_xpath(function_xpath).text.splitlines(0)[1]
        functions.append(job_function)
    except:
        functions.append('')
    
    # Industries
    try:
        industry_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[4]'
        industry_type = driver.find_element_by_xpath(industry_xpath).text.splitlines(0)[1]
        industries.append(industry_type)
    except Exception:
        industries.append('')

In [13]:
# to check if we have all information
print(len(job_id))
print(len(post_date))
print(len(company_name))
print(len(post_title))
print(len(job_location))
print(len(job_desc))
print(len(level))
print(len(emp_type))
print(len(functions))
print(len(industries))

600
600
600
600
600
600
600
600
600
600


In [14]:
job_data = pd.DataFrame({
'Job ID': job_id,
'Date': post_date,
'Company Name': company_name,
'Post': post_title,
'Location': job_location,
'Applicants': appl_num,
'Description': job_desc,
'Level': level,
'Type': emp_type,
'Function': functions,
'Industry': industries
})

In [15]:
job_data

,Job ID,Date,Company Name,Post,Location,Applicants,Description,Level,Type,Function,Industry
0,2174297164,2020-09-25,Drogaria Araújo,Analista de Atração e Seleção,"Belo Horizonte, Minas Gerais, Brazil",25,Atuar com as rotinas do Recrutamento e Seleção...,Entry level,Full-time,Business DevelopmentSales,Staffing and RecruitingHuman ResourcesRetail
1,2166662774,2020-09-17,Hermes Pardini,Analista de RH Jr - Vaga Temporária,"Belo Horizonte, Minas Gerais, Brazil",25,O que é necessário que essa pessoa já tenha de...,Associate,Full-time,Human Resources,Staffing and RecruitingHuman ResourcesRetail
2,2174300513,2020-09-25,Pimenta Associados,Coordenador de recursos humanos,"Belo Horizonte, Minas Gerais, Brazil",25,Desenvolver o setor atuando forte no turnover ...,Entry level,Full-time,Human Resources,Staffing and RecruitingHuman ResourcesManageme...
3,2168867662,2020-09-16,Líder Aviação,Analista de Recursos Humanos (Foco recrutament...,"Belo Horizonte, Minas Gerais, Brazil",66 applicants,Requisitos Obrigatórios\nFormação superior com...,Associate,Full-time,Human Resources,Aviation & AerospaceHuman ResourcesAirlines/Av...
4,2164126242,2020-09-16,Paola Salgado,Estágio Em RH,"Belo Horizonte, Minas Gerais, Brazil",48 applicants,"Vaga de Estágio Em RH em Minas Gerais. R$ 900,...",Entry level,Full-time,Human Resources,Information Technology and ServicesStaffing an...
...,...,...,...,...,...,...,...,...,...,...,...
595,2160163074,2020-09-12,Behaveconsulting,Analista de Recrutamento e Seleção,"Lençóis Paulista, São Paulo, Brazil",25,Behave assessorando indústria de grande porte ...,Entry level,Full-time,Human Resources,Marketing and AdvertisingInformation Technolog...
596,1627342581,2019-11-28,People TI,Analista para Assessment - Banco de Talentos,"Florianópolis, Santa Catarina, Brazil",101 applicants,Nível: Analista\nÁrea: Recursos Humanos\nEscol...,Mid-Senior level,Full-time,Business DevelopmentSales,Information Technology and ServicesComputer So...
597,2006541359,2020-08-02,Escola Moppe | Banco de Talentos,Coordenador Pedagógico de Ensino Fundamental I,"São José dos Campos, São Paulo, Brazil",25,Esta é uma vaga de Banco de Talentos. Conheça ...,Entry level,Full-time,EducationTraining,Education Management
598,2172386517,2020-09-24,Eleve Desenvolvimento Humano,Analista de rh - irá atuar em papelaria em vár...,"Várzea Grande, Mato Grosso, Brazil",25,Requisitos: Ensino Superior Completo (Psicolog...,Associate,Full-time,Human Resources,Computer SoftwareStaffing and RecruitingHuman ...


In [ ]:
# # cleaning description column
# job_data['Description'] = job_data['Description'].str.replace('\n',' ')

In [18]:
job_data.to_csv('psicologia.csv',index=0)